# Atenuadores adaptados de banda ancha y angosta
<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
bla ..
* Funciones de dibujo de redes: [dibujar_puerto_entrada](), [dibujar_funcion_exc_abajo](), [dibujar_elemento_derivacion](), [dibujar_espacio_derivacion](), [dibujar_elemento_serie]()
* Funciones de síntesis de dipolos: [remover_polo_infinito](), [remover_valor_en_infinito]()
* Funciones para presentación de markdown y latex: [print_latex](), [print_subtitle](), [a_equal_b_latex_s]()


### Introducción

En esta sección se comienza con un tema central de la asignatura *Teoría de Circuitos 2* que es la síntesis de redes pasivas que cumplan alguna restricción dada por la función transferencia (FT). Para ello utilizaremos los conceptos de síntesis de funciones de excitación (FE) vistos en la sección anterior. De hecho la metodología se fundamenta en convertir la restricción impuesta por la FT en una FE que pueda sintetizarse siguiendo cualquiera de los métodos vistos.

Este documento se centra en la descripción y uso de la metodología mediante las herramientas computacionales. El lector interesado, puede consultar más detalles en:

1. Kuo, F. *Network Analysis and Synthesis*.
2. Araujo, N. Síntesis de redes.

Para cualquier cuadripolo descargado se sabe que los 4 parámetros transferencia (ABCD) pueden escribirse en función de los demás juegos de parámetros que se definieron con anterioridad. Nos interesará especialmente lograr definir estos parámetros como el cociente de una FT y otra FE. Si se revisa cualquier tabla de conversión de parámetros hallamos, por ejemplo, que:

$$ A = \frac{Z_{11}}{Z_{21}} = -\frac{Y_{22}}{Y_{21}} $$ 
$$ D = \frac{Z_{22}}{Z_{21}} = -\frac{Y_{11}}{Y_{21}} $$ 

Esto significa que para realizar una síntesis de una FT de tensiones A, podremos lograrla tanto a través de $Z_{11}$ o $Y_{22}$. Recordando ambas definiciones, esto resulta en una síntesis *desde* el puerto de entrada o salida respectivamente. Independientemente de la elección, se sabe que $ A = \frac{P(s)}{Q(s)} $ es una función racional y tanto $Z_{11}$ como $Z_{21}$, por lo general, no podrán ser solo polinomios P y Q. Por lo tanto conviene adoptar un polinomio auxiliar $R(s)$, de forma que

$$ A = \frac{\frac{P}{R}}{\frac{Q}{R}} $$

y asegurando que $Z_{11} = \frac{P}{R}$ sea **FRP** y $Z_{21} = \frac{Q}{R}$ sea **estable**. Lo mismo puede decirse en caso que se eligiera $Y_{22}$. Como puede notarse, al sintetizar $Z_{11}$ solo cumpliríamos con el polinomio $P(s)$, a menos que utilicemos el polinomio $Q(s)$ como guía para realizar **las remociones donde $Q(s)$ tenga raíces**. Por último decir que si se asegura que la FE sea FRP ($Z_{11}$ o $Y_{22}$) al mismo se garantiza que la FT sea estable. 

Conviene recordar que por lo general las FT se definen como el recíproco de A y del resto de los parámetros ABCD. Es decir 

$$ \frac{V_2}{V_1} = \frac{1}{A} = T(s) $$.

Por lo tanto, es equivalente decir que al sintetizar $Z_{11}$ se cumple **solo** con los polos de $T$, mientras que al remover los polos de $Z_{11}$ en las frecuencias de los ceros de $Z_{21}$ (o $T$), **se cumplirá con la restricción impuesta por T(s) en su conjunto**.

Como es habitual se realizarán algunos ejemplos para ilustrar los conceptos. Se inicializa el entorno de trabajo para comenzar con el primer ejemplo

In [1]:

import sympy as sp

# Ahora importamos las funciones de PyTC2

from pytc2.remociones import remover_polo_dc, remover_polo_infinito, isFRP
from pytc2.remociones import remover_valor_en_dc, remover_valor_en_infinito, remover_polo_sigma

from pytc2.sintesis_dipolo import cauer_RC, foster, foster_zRC2yRC
from pytc2.dibujar import dibujar_cauer_RC_RL, dibujar_foster_derivacion, dibujar_foster_serie
from pytc2.general import print_latex, print_subtitle, a_equal_b_latex_s
from IPython.display import display,  Markdown


## Ejemplo 1:

Sea la siguiente función de impedancia:

$$ Z(s) = \frac{(s^2+2s+2)}{(s^2+s+1)} $$ 

Se comienza por la inicialización de variables y la función dada